In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LassoCV, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, brier_score_loss
from tqdm import tqdm
from xgboost import XGBClassifier

In [ ]:
scaler = StandardScaler()

VIX = pd.read_csv('../01 Raw Data/Others/VIXTWN.csv', index_col='Date', parse_dates=['Date'])
VIX[:] = scaler.fit_transform(VIX.values)

MI = pd.read_csv('../01 Raw Data/Others/MI.csv', index_col='Date', parse_dates=['Date'])
MI['Recession'] = MI['MI'].apply(lambda x: 1 if x <= 16 else 0)

In [ ]:
datasets = ['Lambda', 'VIX', 'FRM']
features = ['ALL', 'LASSO']
companies = ['Cap_63', 'Cap_126', 'Ele_63', 'Ele_126', 'Fin_63', 'Fin_126', 'FinEle_63', 'FinEle_126']

# Lambda

In [ ]:
dataset = 'Lambda'

for feature in features:
    companies = ['Cap_63', 'Cap_126', 'Ele_63', 'Ele_126', 'Fin_63', 'Fin_126', 'FinEle_63', 'FinEle_126']
    for company in companies:
        sector, ws = company.split('_')[0], int(company.split('_')[1])

        print(f'{dataset}, Feature = {feature}, {company}')
        top = 20
        lag_ws = 63
        input_path1 = f'../03 Modeling/{company}'
        input_path2 = f'../01 Raw Data/{sector}'
        output_path = f'{company}'
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        full_lambda = pd.read_csv(f'{input_path1}/full_lambda.csv', index_col='Date', parse_dates=['Date'])
        stock_rank = pd.read_csv(f'{input_path2}/Top rank company.csv', index_col='Date', parse_dates=['Date'])
        stock_rank = stock_rank.astype(str)
        lambda_rank = pd.DataFrame(index=full_lambda.index, columns=range(1, 21))
        for date in full_lambda.index:
            for col in lambda_rank.columns:
                company_id = str(stock_rank.loc[date, str(col)]).split('.')[0]
                lambda_rank.loc[date, col] = full_lambda.loc[date, company_id]
        scaler = MinMaxScaler()
        lambda_rank[:] = scaler.fit_transform(lambda_rank.values)
        lambda_rank_L = pd.DataFrame()
        for column in lambda_rank:
            lagged_cols = {f'{column}_{lag}': lambda_rank.loc[:,column].shift(lag) for lag in range(round(lag_ws+1))}
            lagged = pd.concat(lagged_cols, axis=1).dropna().resample('M').last()
            lambda_rank_L = pd.concat([lambda_rank_L, lagged], axis=1)
        MI_filter = MI.loc[MI.index.to_series().apply(lambda x: pd.Period(x, freq='M')).isin(lambda_rank_L.index.to_period('M').unique())]
        X = lambda_rank_L.copy()

        y = MI_filter['Recession']

        for col in X.columns:
            X[col] = pd.to_numeric(X[col], errors='coerce')

        cut = 5
        n = len(X)

        models = {
            "Logistic Regression": LogisticRegression(max_iter=10000, random_state=42, multi_class='ovr'),
            "XGBoost": XGBClassifier(eval_metric='mlogloss', random_state=42)
        }

        results = pd.DataFrame(columns=["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"])
        all_predictions = {model_name: pd.Series([None] * n, index=MI_filter.index) for model_name in models.keys()}

        n_samples = X.shape[0]
        split_size = n_samples // cut

        for i in tqdm(range(cut)):
            start_test = i * split_size
            end_test = (i + 1) * split_size if i < cut - 1 else n_samples

            X_test = X[start_test:end_test]
            y_test = y[start_test:end_test]

            indices = list(range(0, start_test)) + list(range(end_test, n_samples))
            X_train = X.iloc[indices]
            y_train = y.iloc[indices]

            if feature == 'LASSO':
                model = LogisticRegression(penalty='l1', solver='saga', max_iter=100000, random_state=42)
                model.fit(X_train, y_train)
                selected_features = np.where(model.coef_[0] != 0)[0]
                X_train = X_train.iloc[:, selected_features]
                X_test = X_test.iloc[:, selected_features]

            row_results = []

            for name, model in models.items():
                model.fit(X_train, y_train)
                
                y_pred_train = model.predict(X_train)
                y_pred_test = model.predict(X_test)
                y_pred_proba_train = model.predict_proba(X_train)[:, 1]
                y_pred_proba_test = model.predict_proba(X_test)[:, 1]

                auc_score_train = roc_auc_score(y_train, y_pred_proba_train)
                brier_score_train = brier_score_loss(y_train, y_pred_proba_train)
                auc_score_test = roc_auc_score(y_test, y_pred_proba_test)
                brier_score_test = brier_score_loss(y_test, y_pred_proba_test)

                row_results.extend([round(auc_score_train, 4), round(brier_score_train, 4), round(auc_score_test, 4), round(brier_score_test, 4)])

                all_predictions[name].iloc[start_test:end_test] = y_pred_proba_test

                fig, ax = plt.subplots(figsize=(20, 4))
                color = 'darkgoldenrod' if name == 'Logistic Regression' else 'darkblue'
                ax.scatter(MI_filter.index[indices], y_train, label='Actual', color='darkred', marker='s', s=14)
                ax.plot(MI_filter.index[indices], y_pred_proba_train, label=f'Predicted by {name}', linewidth=4, color=color, alpha=0.2)
                count = 0
                for idx in indices:
                    if y_train.loc[MI_filter.index[idx]] == 1 and y_pred_proba_train[count] < 0.5:
                        ax.scatter(MI_filter.index[idx], y_pred_proba_train[count], color='darkgreen', marker='x', s=50, lw=2.5, alpha=1)
                    elif y_train.loc[MI_filter.index[idx]] == 0 and y_pred_proba_train[count] > 0.5:
                        ax.scatter(MI_filter.index[idx], y_pred_proba_train[count], color='darkgreen', marker='^', s=40, lw=1, alpha=1)
                    else:
                        ax.scatter(MI_filter.index[idx], y_pred_proba_train[count], color=color, alpha=1)
                    count += 1
                ax.axhline(y=0.5, color='k', linestyle='--', alpha=0.4)
                ax.set_xlim(y.index[0], y.index[-1])
                ax.set_ylim(-0.1, 1.1)
                ax.tick_params(axis='y', labelsize=8, width=2)
                ax.tick_params(axis='x', labelsize=8, width=2)
                ax.xaxis.set_major_locator(mdates.YearLocator())
                ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
                plt.tight_layout()
                plt.savefig(f'{output_path}/IN {dataset} {feature} {name} F{i+1}.png', dpi=144, transparent=True)
                plt.close()

            results.loc[i] = row_results

        results.columns = ["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"]

        average_results = results.mean().to_frame().T
        average_results.index = ["-"]

        print(results.round(4))
        print(average_results.round(4))
        print()

        for name in models.keys():
            plt.figure(figsize=(20, 4))
            color = 'darkgoldenrod' if name == 'Logistic Regression' else 'darkblue'
            plt.scatter(MI_filter.index, y, label='Actual', color='darkred', marker='s', s=14)
            plt.plot(MI_filter.index, all_predictions[name], label=f'Predicted by {name}', linewidth=4, color=color, alpha=0.2)
            for i in range(len(all_predictions[name])):
                if y.iloc[i] == 1 and all_predictions[name].values[i] < 0.5:
                    plt.scatter(all_predictions[name].index[i], all_predictions[name].values[i], color='darkgreen', marker='x', s=50, lw=2.5, alpha=1)
                elif y.iloc[i] == 0 and all_predictions[name].values[i] > 0.5:
                    plt.scatter(all_predictions[name].index[i], all_predictions[name].values[i], color='darkgreen', marker='^', s=40, lw=1, alpha=1)
                else:
                    plt.scatter(all_predictions[name].index[i], all_predictions[name].values[i], color=color, alpha=1)

            plt.axhline(y=0.5, color='k', linestyle='--', alpha=0.4)
            plt.ylim(-0.1, 1.1)
            plt.yticks(fontsize=14, fontweight='heavy')
            plt.xticks(fontsize=14, fontweight='heavy')
            plt.gca().xaxis.set_major_locator(mdates.YearLocator())
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
            plt.tight_layout()
            plt.savefig(f'{output_path}/OUT {dataset} {feature} {name}.png', dpi=144, transparent=True)
            plt.close()


# FRM

In [ ]:
dataset = 'FRM'

for feature in features:
    companies = ['Cap_63', 'Cap_126', 'Ele_63', 'Ele_126', 'Fin_63', 'Fin_126', 'FinEle_63', 'FinEle_126']
    for company in companies:
        sector, ws = company.split('_')[0], int(company.split('_')[1])

        print(f'{company}, Feature = {feature}')
        top = 20
        lag_ws = 63
        input_path1 = f'../03 Modeling/{company}'
        input_path2 = f'../01 Raw Data/{sector}'
        output_path = f'{company}'
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        full_lambda = pd.read_csv(f'{input_path1}/full_lambda.csv', index_col='Date', parse_dates=['Date'])
        FRM = full_lambda.mean(axis=1) * full_lambda.shape[1] / top
        lagged_cols = {f'FRM_lag_{lag}': FRM.shift(lag) for lag in range(round(lag_ws+1))}
        FRM_L = pd.concat(lagged_cols, axis=1).dropna().resample('M').last()
        FRM_L = FRM_L.dropna()
        FRM_L = FRM_L.resample('M').last()
        MI_filter = MI.loc[MI.index.to_series().apply(lambda x: pd.Period(x, freq='M')).isin(FRM_L.index.to_period('M').unique())]
        
        X = FRM_L.copy()
        y = MI_filter['Recession']

        for col in X.columns:
            X[col] = pd.to_numeric(X[col], errors='coerce')

        cut = 5
        n = len(X)

        models = {
            "Logistic Regression": LogisticRegression(max_iter=10000, random_state=42, multi_class='ovr'),
            "XGBoost": XGBClassifier(eval_metric='mlogloss', random_state=42)
        }

        results = pd.DataFrame(columns=["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"])
        all_predictions = {model_name: pd.Series([None] * n, index=MI_filter.index) for model_name in models.keys()}

        n_samples = X.shape[0]
        split_size = n_samples // cut

        for i in tqdm(range(cut)):
            start_test = i * split_size
            end_test = (i + 1) * split_size if i < cut - 1 else n_samples

            X_test = X[start_test:end_test]
            y_test = y[start_test:end_test]

            indices = list(range(0, start_test)) + list(range(end_test, n_samples))
            X_train = X.iloc[indices]
            y_train = y.iloc[indices]

            if feature == 'LASSO':
                model = LogisticRegression(penalty='l1', solver='saga', max_iter=100000, random_state=42)
                model.fit(X_train, y_train)
                selected_features = np.where(model.coef_[0] != 0)[0]
                X_train = X_train.iloc[:, selected_features]
                X_test = X_test.iloc[:, selected_features]

            row_results = []

            for name, model in models.items():
                model.fit(X_train, y_train)
                
                y_pred_train = model.predict(X_train)
                y_pred_test = model.predict(X_test)
                y_pred_proba_train = model.predict_proba(X_train)[:, 1]
                y_pred_proba_test = model.predict_proba(X_test)[:, 1]

                auc_score_train = roc_auc_score(y_train, y_pred_proba_train)
                brier_score_train = brier_score_loss(y_train, y_pred_proba_train)
                auc_score_test = roc_auc_score(y_test, y_pred_proba_test)
                brier_score_test = brier_score_loss(y_test, y_pred_proba_test)

                row_results.extend([round(auc_score_train, 4), round(brier_score_train, 4), round(auc_score_test, 4), round(brier_score_test, 4)])

                all_predictions[name].iloc[start_test:end_test] = y_pred_proba_test

                fig, ax = plt.subplots(figsize=(20, 4))
                color = 'darkgoldenrod' if name == 'Logistic Regression' else 'darkblue'
                ax.scatter(MI_filter.index[indices], y_train, label='Actual', color='darkred', marker='s', s=14)
                ax.plot(MI_filter.index[indices], y_pred_proba_train, label=f'Predicted by {name}', linewidth=4, color=color, alpha=0.2)
                count = 0
                for idx in indices:
                    if y_train.loc[MI_filter.index[idx]] == 1 and y_pred_proba_train[count] < 0.5:
                        ax.scatter(MI_filter.index[idx], y_pred_proba_train[count], color='darkgreen', marker='x', s=50, lw=2.5, alpha=1)
                    elif y_train.loc[MI_filter.index[idx]] == 0 and y_pred_proba_train[count] > 0.5:
                        ax.scatter(MI_filter.index[idx], y_pred_proba_train[count], color='darkgreen', marker='^', s=40, lw=1, alpha=1)
                    else:
                        ax.scatter(MI_filter.index[idx], y_pred_proba_train[count], color=color, alpha=1)
                    count += 1
                ax.axhline(y=0.5, color='k', linestyle='--', alpha=0.4)
                ax.set_xlim(y.index[0], y.index[-1])
                ax.set_ylim(-0.1, 1.1)
                ax.tick_params(axis='y', labelsize=8, width=2)
                ax.tick_params(axis='x', labelsize=8, width=2)
                ax.xaxis.set_major_locator(mdates.YearLocator())
                ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
                plt.tight_layout()
                plt.savefig(f'{output_path}/IN {dataset} {feature} {name} F{i+1}.png', dpi=144, transparent=True)
                plt.close()

            results.loc[i] = row_results

        results.columns = ["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"]

        average_results = results.mean().to_frame().T
        average_results.index = ["-"]

        print(results.round(4))
        print(average_results.round(4))
        print()

        for name in models.keys():
            plt.figure(figsize=(20, 4))
            color = 'darkgoldenrod' if name == 'Logistic Regression' else 'darkblue'
            plt.scatter(MI_filter.index, y, label='Actual', color='darkred', marker='s', s=14)
            plt.plot(MI_filter.index, all_predictions[name], label=f'Predicted by {name}', linewidth=4, color=color, alpha=0.2)
            for i in range(len(all_predictions[name])):
                if y.iloc[i] == 1 and all_predictions[name].values[i] < 0.5:
                    plt.scatter(all_predictions[name].index[i], all_predictions[name].values[i], color='darkgreen', marker='x', s=50, lw=2.5, alpha=1)
                elif y.iloc[i] == 0 and all_predictions[name].values[i] > 0.5:
                    plt.scatter(all_predictions[name].index[i], all_predictions[name].values[i], color='darkgreen', marker='^', s=40, lw=1, alpha=1)
                else:
                    plt.scatter(all_predictions[name].index[i], all_predictions[name].values[i], color=color, alpha=1)

            plt.axhline(y=0.5, color='k', linestyle='--', alpha=0.4)
            plt.ylim(-0.1, 1.1)
            plt.yticks(fontsize=14, fontweight='heavy')
            plt.xticks(fontsize=14, fontweight='heavy')
            plt.gca().xaxis.set_major_locator(mdates.YearLocator())
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
            plt.tight_layout()
            plt.savefig(f'{output_path}/OUT {dataset} {feature} {name}.png', dpi=144, transparent=True)
            plt.close()


# VIX

In [ ]:
dataset = 'VIX'

for feature in features:
    for lag_ws in [63, 126, 189]:
        print(f'{dataset}, Feature = {feature}')
        top = 20
        input_path1 = f'../03 Modeling/{company}'
        input_path2 = f'../01 Raw Data/{sector}'
        output_path = 'VIX'
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        lagged_cols = {f'VIX_lag_{lag}': VIX['VIXTWN'].shift(lag) for lag in range(round(lag_ws+1))}
        VIX_L = pd.concat(lagged_cols, axis=1).dropna().resample('M').last()
        VIX_L = VIX_L.dropna()
        VIX_L = VIX_L.resample('M').last()
        MI_filter = MI.loc[MI.index.to_series().apply(lambda x: pd.Period(x, freq='M')).isin(VIX_L.index.to_period('M').unique())]
        
        X = VIX_L.copy()
        y = MI_filter['Recession']

        for col in X.columns:
            X[col] = pd.to_numeric(X[col], errors='coerce')

        cut = 5
        n = len(X)

        models = {
            "Logistic Regression": LogisticRegression(max_iter=10000, random_state=42, multi_class='ovr'),
            "XGBoost": XGBClassifier(eval_metric='mlogloss', random_state=42)
        }

        results = pd.DataFrame(columns=["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"])
        all_predictions = {model_name: pd.Series([None] * n, index=MI_filter.index) for model_name in models.keys()}

        n_samples = X.shape[0]
        split_size = n_samples // cut

        for i in tqdm(range(cut)):
            start_test = i * split_size
            end_test = (i + 1) * split_size if i < cut - 1 else n_samples

            X_test = X[start_test:end_test]
            y_test = y[start_test:end_test]

            indices = list(range(0, start_test)) + list(range(end_test, n_samples))
            X_train = X.iloc[indices]
            y_train = y.iloc[indices]

            if feature == 'LASSO':
                model = LogisticRegression(penalty='l1', solver='saga', max_iter=100000, random_state=42)
                model.fit(X_train, y_train)
                selected_features = np.where(model.coef_[0] != 0)[0]
                X_train = X_train.iloc[:, selected_features]
                X_test = X_test.iloc[:, selected_features]

            row_results = []

            for name, model in models.items():
                model.fit(X_train, y_train)
                
                y_pred_train = model.predict(X_train)
                y_pred_test = model.predict(X_test)
                y_pred_proba_train = model.predict_proba(X_train)[:, 1]
                y_pred_proba_test = model.predict_proba(X_test)[:, 1]

                auc_score_train = roc_auc_score(y_train, y_pred_proba_train)
                brier_score_train = brier_score_loss(y_train, y_pred_proba_train)
                auc_score_test = roc_auc_score(y_test, y_pred_proba_test)
                brier_score_test = brier_score_loss(y_test, y_pred_proba_test)

                row_results.extend([round(auc_score_train, 4), round(brier_score_train, 4), round(auc_score_test, 4), round(brier_score_test, 4)])

                all_predictions[name].iloc[start_test:end_test] = y_pred_proba_test

                fig, ax = plt.subplots(figsize=(20, 4))
                color = 'darkgoldenrod' if name == 'Logistic Regression' else 'darkblue'
                ax.scatter(MI_filter.index[indices], y_train, label='Actual', color='darkred', marker='s', s=14)
                ax.plot(MI_filter.index[indices], y_pred_proba_train, label=f'Predicted by {name}', linewidth=4, color=color, alpha=0.2)
                count = 0
                for idx in indices:
                    if y_train.loc[MI_filter.index[idx]] == 1 and y_pred_proba_train[count] < 0.5:
                        ax.scatter(MI_filter.index[idx], y_pred_proba_train[count], color='darkgreen', marker='x', s=50, lw=2.5, alpha=1)
                    elif y_train.loc[MI_filter.index[idx]] == 0 and y_pred_proba_train[count] > 0.5:
                        ax.scatter(MI_filter.index[idx], y_pred_proba_train[count], color='darkgreen', marker='^', s=40, lw=1, alpha=1)
                    else:
                        ax.scatter(MI_filter.index[idx], y_pred_proba_train[count], color=color, alpha=1)
                    count += 1
                ax.axhline(y=0.5, color='k', linestyle='--', alpha=0.4)
                ax.set_xlim(y.index[0], y.index[-1])
                ax.set_ylim(-0.1, 1.1)
                ax.tick_params(axis='y', labelsize=8, width=2)
                ax.tick_params(axis='x', labelsize=8, width=2)
                ax.xaxis.set_major_locator(mdates.YearLocator())
                ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
                plt.tight_layout()
                plt.savefig(f'{output_path}/IN {dataset} {lag_ws} {feature} {name} F{i+1}.png', dpi=144, transparent=True)
                plt.close()

            results.loc[i] = row_results

        results.columns = ["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"]

        average_results = results.mean().to_frame().T
        average_results.index = ["-"]

        print(results.round(4))
        print(average_results.round(4))
        print()

        for name in models.keys():
            plt.figure(figsize=(20, 4))
            color = 'darkgoldenrod' if name == 'Logistic Regression' else 'darkblue'
            plt.scatter(MI_filter.index, y, label='Actual', color='darkred', marker='s', s=14)
            plt.plot(MI_filter.index, all_predictions[name], label=f'Predicted by {name}', linewidth=4, color=color, alpha=0.2)
            for i in range(len(all_predictions[name])):
                if y.iloc[i] == 1 and all_predictions[name].values[i] < 0.5:
                    plt.scatter(all_predictions[name].index[i], all_predictions[name].values[i], color='darkgreen', marker='x', s=50, lw=2.5, alpha=1)
                elif y.iloc[i] == 0 and all_predictions[name].values[i] > 0.5:
                    plt.scatter(all_predictions[name].index[i], all_predictions[name].values[i], color='darkgreen', marker='^', s=40, lw=1, alpha=1)
                else:
                    plt.scatter(all_predictions[name].index[i], all_predictions[name].values[i], color=color, alpha=1)

            plt.axhline(y=0.5, color='k', linestyle='--', alpha=0.4)
            plt.ylim(-0.1, 1.1)
            plt.yticks(fontsize=14, fontweight='heavy')
            plt.xticks(fontsize=14, fontweight='heavy')
            plt.gca().xaxis.set_major_locator(mdates.YearLocator())
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
            plt.tight_layout()
            plt.savefig(f'{output_path}/OUT {dataset} {lag_ws} {feature} {name}.png', dpi=144, transparent=True)
            plt.close()


# Lambdas, without Feature selection

In [ ]:
feature = 'ALL'

companies = ['Cap_63', 'Cap_126', 'Ele_63', 'Ele_126', 'Fin_63', 'Fin_126', 'FinEle_63', 'FinEle_126']
company = 'Cap_63'
for company in companies:
    sector, ws = company.split('_')[0], int(company.split('_')[1])

    print(company)
    top = 20
    lag_ws = 63
    input_path1 = f'../03 Modeling/{company}'
    input_path2 = f'../01 Raw Data/{sector}'
    output_path = f'{company}'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    full_lambda = pd.read_csv(f'{input_path1}/full_lambda.csv', index_col='Date', parse_dates=['Date'])
    stock_rank = pd.read_csv(f'{input_path2}/Top rank company.csv', index_col='Date', parse_dates=['Date'])
    stock_rank = stock_rank.astype(str)

    lambda_rank = pd.DataFrame(index=full_lambda.index, columns=range(1, 21))
    for date in full_lambda.index:
        for col in lambda_rank.columns:
            company_id = str(stock_rank.loc[date, str(col)]).split('.')[0]
            lambda_rank.loc[date, col] = full_lambda.loc[date, company_id]
    scaler = MinMaxScaler()
    lambda_rank[:] = scaler.fit_transform(lambda_rank.values)
    lambda_rank_L = pd.DataFrame()
    for column in lambda_rank:
        lagged_cols = {f'{column}_{lag}': lambda_rank.loc[:,column].shift(lag) for lag in range(round(lag_ws+1))}
        lagged = pd.concat(lagged_cols, axis=1).dropna().resample('M').last()
        lambda_rank_L = pd.concat([lambda_rank_L, lagged], axis=1)

    MI_filter = MI.loc[MI.index.to_series().apply(lambda x: pd.Period(x, freq='M')).isin(lambda_rank_L.index.to_period('M').unique())]

    #####

    X = lambda_rank_L.copy()
    y = MI_filter['Recession']

    for col in X.columns:
        X[col] = pd.to_numeric(X[col], errors='coerce')

    cut = 5
    n = len(X)

    models = {
        "LR": LogisticRegression(max_iter=10000, random_state=42, multi_class='ovr'),   # Logistic Regression
        "XGB": XGBClassifier(eval_metric='mlogloss', random_state=42)
    }

    results = pd.DataFrame(columns=["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"])
    all_predictions = {model_name: pd.Series([None] * n, index=MI_filter.index) for model_name in models.keys()}

    n_samples = X.shape[0]
    split_size = n_samples // cut

    for i in tqdm(range(cut)):
        start_test = i * split_size
        end_test = (i + 1) * split_size if i < cut - 1 else n_samples

        X_test = X[start_test:end_test]
        y_test = y[start_test:end_test]

        indices = list(range(0, start_test)) + list(range(end_test, n_samples))
        X_train = X.iloc[indices]
        y_train = y.iloc[indices]

        row_results = []

        for name, model in models.items():
            model.fit(X_train, y_train)
            
            y_pred_train = model.predict(X_train)
            y_pred_test = model.predict(X_test)
            y_pred_proba_train = model.predict_proba(X_train)[:, 1]
            y_pred_proba_test = model.predict_proba(X_test)[:, 1]

            auc_score_train = roc_auc_score(y_train, y_pred_proba_train)
            brier_score_train = brier_score_loss(y_train, y_pred_proba_train)
            auc_score_test = roc_auc_score(y_test, y_pred_proba_test)
            brier_score_test = brier_score_loss(y_test, y_pred_proba_test)

            row_results.extend([round(auc_score_train, 4), round(brier_score_train, 4), round(auc_score_test, 4), round(brier_score_test, 4)])

            all_predictions[name].iloc[start_test:end_test] = y_pred_proba_test

            fig, ax = plt.subplots(figsize=(20, 4))
            color = 'darkgoldenrod' if name == 'Logistic Regression' else 'darkblue'
            ax.scatter(MI_filter.index[indices], y_train, label='Actual', color='darkred', marker='s', s=14)
            ax.plot(MI_filter.index[indices], y_pred_proba_train, label=f'Predicted by {name}', linewidth=4, color=color, alpha=0.2)
            count = 0
            for idx in indices:
                if y_train.loc[MI_filter.index[idx]] == 1 and y_pred_proba_train[count] < 0.5:
                    ax.scatter(MI_filter.index[idx], y_pred_proba_train[count], color='darkgreen', marker='x', s=50, lw=2.5, alpha=1)
                elif y_train.loc[MI_filter.index[idx]] == 0 and y_pred_proba_train[count] > 0.5:
                    ax.scatter(MI_filter.index[idx], y_pred_proba_train[count], color='darkgreen', marker='^', s=40, lw=1, alpha=1)
                else:
                    ax.scatter(MI_filter.index[idx], y_pred_proba_train[count], color=color, alpha=1)
                count += 1
            ax.axhline(y=0.5, color='k', linestyle='--', alpha=0.4)
            ax.set_xlim(y.index[0], y.index[-1])
            ax.set_ylim(-0.1, 1.1)
            ax.tick_params(axis='y', labelsize=8, width=2)
            ax.tick_params(axis='x', labelsize=8, width=2)
            ax.xaxis.set_major_locator(mdates.YearLocator())
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
            plt.tight_layout()
            plt.savefig(f'{output_path}/IN {feature} {name} F{i+1}.png', dpi=144, transparent=True)
            plt.close()

        results.loc[i] = row_results

    results.columns = ["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"]

    average_results = results.mean().to_frame().T
    average_results.index = ["-"]

    print(results.round(4))
    print(average_results.round(4))
    print()

    for name in models.keys():
        plt.figure(figsize=(20, 4))
        color = 'darkgoldenrod' if name == 'Logistic Regression' else 'darkblue'
        plt.scatter(MI_filter.index, y, label='Actual', color='darkred', marker='s', s=14)
        # plt.scatter(MI_filter.index, all_predictions[name], label=f'Predicted by {name}', s=20)
        plt.plot(MI_filter.index, all_predictions[name], label=f'Predicted by {name}', linewidth=4, color=color, alpha=0.2)
        for i in range(len(all_predictions[name])):
            if y.iloc[i] == 1 and all_predictions[name].values[i] < 0.5:
                plt.scatter(all_predictions[name].index[i], all_predictions[name].values[i], color='darkgreen', marker='x', s=50, lw=2.5, alpha=1)
            elif y.iloc[i] == 0 and all_predictions[name].values[i] > 0.5:
                plt.scatter(all_predictions[name].index[i], all_predictions[name].values[i], color='darkgreen', marker='^', s=40, lw=1, alpha=1)
            else:
                plt.scatter(all_predictions[name].index[i], all_predictions[name].values[i], color=color, alpha=1)

        plt.axhline(y=0.5, color='k', linestyle='--', alpha=0.4)
        plt.ylim(-0.1, 1.1)
        plt.yticks(fontsize=14, fontweight='heavy')
        plt.xticks(fontsize=14, fontweight='heavy')
        plt.gca().xaxis.set_major_locator(mdates.YearLocator())
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        plt.tight_layout()
        plt.savefig(f'{output_path}/OUT {feature} {name}.png', dpi=144, transparent=True)
        plt.close()

# Lambdas, with Feature Selection

In [ ]:
companies = ['Cap_63', 'Cap_126', 'Ele_63', 'Ele_126', 'Fin_63', 'Fin_126', 'FinEle_63', 'FinEle_126']
company = 'Cap_63'
for company in companies:
    sector, ws = company.split('_')[0], int(company.split('_')[1])

    print(company)
    top = 20
    lag_ws = 63
    input_path1 = f'../03 Modeling/{company}'
    input_path2 = f'../01 Raw Data/{sector}'
    output_path = f'{company}'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    full_lambda = pd.read_csv(f'{input_path1}/full_lambda.csv', index_col='Date', parse_dates=['Date'])
    stock_rank = pd.read_csv(f'{input_path2}/Top rank company.csv', index_col='Date', parse_dates=['Date'])
    stock_rank = stock_rank.astype(str)

    lambda_rank = pd.DataFrame(index=full_lambda.index, columns=range(1, 21))
    for date in full_lambda.index:
        for col in lambda_rank.columns:
            company_id = str(stock_rank.loc[date, str(col)]).split('.')[0]
            lambda_rank.loc[date, col] = full_lambda.loc[date, company_id]
    scaler = MinMaxScaler()
    lambda_rank[:] = scaler.fit_transform(lambda_rank.values)
    lambda_rank_L = pd.DataFrame()
    for column in lambda_rank:
        lagged_cols = {f'{column}_{lag}': lambda_rank.loc[:,column].shift(lag) for lag in range(round(lag_ws+1))}
        lagged = pd.concat(lagged_cols, axis=1).dropna().resample('M').last()
        lambda_rank_L = pd.concat([lambda_rank_L, lagged], axis=1)

    MI_filter = MI.loc[MI.index.to_series().apply(lambda x: pd.Period(x, freq='M')).isin(lambda_rank_L.index.to_period('M').unique())]

    #####

    X = lambda_rank_L.copy()
    y = MI_filter['Recession']

    for col in X.columns:
        X[col] = pd.to_numeric(X[col], errors='coerce')

    cut = 5
    n = len(X)

    models = {
        "Logistic Regression": LogisticRegression(max_iter=10000, random_state=42, multi_class='ovr'),
        "XGBoost": XGBClassifier(eval_metric='mlogloss', random_state=42)
    }

    results = pd.DataFrame(columns=["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"])
    all_predictions = {model_name: pd.Series([None] * n, index=MI_filter.index) for model_name in models.keys()}

    n_samples = X.shape[0]
    split_size = n_samples // cut

    for i in tqdm(range(cut)):
        start_test = i * split_size
        end_test = (i + 1) * split_size if i < cut - 1 else n_samples

        X_test = X[start_test:end_test]
        y_test = y[start_test:end_test]

        indices = list(range(0, start_test)) + list(range(end_test, n_samples))
        X_train = X.iloc[indices]
        y_train = y.iloc[indices]

        lasso_cv = LassoCV(cv=10, random_state=42, max_iter=10000, n_jobs=-1).fit(X_train, y_train)
        best_alpha = lasso_cv.alpha_
        lasso = Lasso(alpha=best_alpha)
        lasso.fit(X_train, y_train)

        selected_features = np.where(lasso.coef_ != 0)[0]

        count = 0
        while len(selected_features) < top:
            count += 1
            lasso = Lasso(alpha=best_alpha*(0.5**count))
            lasso.fit(X_train, y_train)
            selected_features = np.where(lasso.coef_ != 0)[0]

        X_train = X_train.iloc[:, selected_features]
        X_test = X_test.iloc[:, selected_features]

        row_results = []

        for name, model in models.items():
            model.fit(X_train, y_train)
            
            y_pred_train = model.predict(X_train)
            y_pred_test = model.predict(X_test)
            y_pred_proba_train = model.predict_proba(X_train)[:, 1]
            y_pred_proba_test = model.predict_proba(X_test)[:, 1]

            auc_score_train = roc_auc_score(y_train, y_pred_proba_train)
            brier_score_train = brier_score_loss(y_train, y_pred_proba_train)
            auc_score_test = roc_auc_score(y_test, y_pred_proba_test)
            brier_score_test = brier_score_loss(y_test, y_pred_proba_test)

            row_results.extend([round(auc_score_train, 4), round(brier_score_train, 4), round(auc_score_test, 4), round(brier_score_test, 4)])

            all_predictions[name].iloc[start_test:end_test] = y_pred_proba_test

        results.loc[i] = row_results

    results.columns = ["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"]

    average_results = results.mean().to_frame().T
    average_results.index = ["-"]

    print(results.round(4))
    print(average_results.round(4))
    print()


# Benchmark: VIXTWN without Feature Selection

In [ ]:
lag_ws = 189

VIX_L = pd.DataFrame()
lagged_cols = {f'VIX_lag_{lag}': VIX['VIXTWN'].shift(lag) for lag in range(round(lag_ws+1))}
VIX_L = pd.concat(lagged_cols, axis=1).dropna().resample('M').last()
VIX_L = VIX_L.dropna()
VIX_L = VIX_L.resample('M').last()

VIX_L = VIX_L.loc[VIX_L.index.intersection(lambda_rank_L.index)]

#####

X = VIX_L.copy()
y = MI_filter['Recession']

for col in X.columns:
    X[col] = pd.to_numeric(X[col], errors='coerce')

cut = 5
n = len(X)

models = {
    "Logistic Regression": LogisticRegression(max_iter=10000, random_state=42, multi_class='ovr'),
    "XGBoost": XGBClassifier(eval_metric='mlogloss', random_state=42)
}

results = pd.DataFrame(columns=["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"])
all_predictions = {model_name: pd.Series([None] * n, index=MI_filter.index) for model_name in models.keys()}

n_samples = X.shape[0]
split_size = n_samples // cut

for i in tqdm(range(cut)):
    start_test = i * split_size
    end_test = (i + 1) * split_size if i < cut - 1 else n_samples

    X_test = X[start_test:end_test]
    y_test = y[start_test:end_test]

    indices = list(range(0, start_test)) + list(range(end_test, n_samples))
    X_train = X.iloc[indices]
    y_train = y.iloc[indices]

    row_results = []

    for name, model in models.items():
        model.fit(X_train, y_train)
        
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        y_pred_proba_train = model.predict_proba(X_train)[:, 1]
        y_pred_proba_test = model.predict_proba(X_test)[:, 1]

        auc_score_train = roc_auc_score(y_train, y_pred_proba_train)
        brier_score_train = brier_score_loss(y_train, y_pred_proba_train)
        auc_score_test = roc_auc_score(y_test, y_pred_proba_test)
        brier_score_test = brier_score_loss(y_test, y_pred_proba_test)

        row_results.extend([round(auc_score_train, 4), round(brier_score_train, 4), round(auc_score_test, 4), round(brier_score_test, 4)])

        all_predictions[name].iloc[start_test:end_test] = y_pred_proba_test

    results.loc[i] = row_results

results.columns = ["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"]

average_results = results.mean().to_frame().T
average_results.index = ["-"]

print(results.round(4))
print(average_results.round(4))
print()


# Benchmark: VIXTWN with Feature Selection

In [ ]:
lag_ws = 189

VIX_L = pd.DataFrame()
lagged_cols = {f'VIX_lag_{lag}': VIX['VIXTWN'].shift(lag) for lag in range(round(lag_ws+1))}
VIX_L = pd.concat(lagged_cols, axis=1).dropna().resample('M').last()
VIX_L = VIX_L.dropna()
VIX_L = VIX_L.resample('M').last()

VIX_L = VIX_L.loc[VIX_L.index.intersection(lambda_rank_L.index)]

#####

X = VIX_L.copy()
y = MI_filter['Recession']

for col in X.columns:
    X[col] = pd.to_numeric(X[col], errors='coerce')

cut = 5
n = len(X)

models = {
    "Logistic Regression": LogisticRegression(max_iter=10000, random_state=42, multi_class='ovr'),
    "XGBoost": XGBClassifier(eval_metric='mlogloss', random_state=42)
}

results = pd.DataFrame(columns=["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"])
all_predictions = {model_name: pd.Series([None] * n, index=MI_filter.index) for model_name in models.keys()}

n_samples = X.shape[0]
split_size = n_samples // cut

for i in tqdm(range(cut)):
    start_test = i * split_size
    end_test = (i + 1) * split_size if i < cut - 1 else n_samples

    X_test = X[start_test:end_test]
    y_test = y[start_test:end_test]

    indices = list(range(0, start_test)) + list(range(end_test, n_samples))
    X_train = X.iloc[indices]
    y_train = y.iloc[indices]

    lasso_cv = LassoCV(cv=5, random_state=42, max_iter=100000, n_jobs=-1).fit(X_train, y_train)
    best_alpha = lasso_cv.alpha_
    lasso = Lasso(alpha=best_alpha)
    lasso.fit(X_train, y_train)

    selected_features = np.where(lasso.coef_ != 0)[0]

    count = 0
    while len(selected_features) < top:
        count += 1
        lasso = Lasso(alpha=best_alpha*(0.5**count))
        lasso.fit(X_train, y_train)
        selected_features = np.where(lasso.coef_ != 0)[0]

    X_train = X_train.iloc[:, selected_features]
    X_test = X_test.iloc[:, selected_features]

    row_results = []

    for name, model in models.items():
        model.fit(X_train, y_train)
        
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        y_pred_proba_train = model.predict_proba(X_train)[:, 1]
        y_pred_proba_test = model.predict_proba(X_test)[:, 1]

        auc_score_train = roc_auc_score(y_train, y_pred_proba_train)
        brier_score_train = brier_score_loss(y_train, y_pred_proba_train)
        auc_score_test = roc_auc_score(y_test, y_pred_proba_test)
        brier_score_test = brier_score_loss(y_test, y_pred_proba_test)

        row_results.extend([round(auc_score_train, 4), round(brier_score_train, 4), round(auc_score_test, 4), round(brier_score_test, 4)])

        all_predictions[name].iloc[start_test:end_test] = y_pred_proba_test

    results.loc[i] = row_results

results.columns = ["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"]

average_results = results.mean().to_frame().T
average_results.index = ["-"]

print(results.round(4))
print(average_results.round(4))
print()

# FRM, without Feature selection

In [ ]:
companies = ['Cap_63', 'Cap_126', 'Ele_63', 'Ele_126', 'Fin_63', 'Fin_126', 'FinEle_63', 'FinEle_126']
company = 'Cap_63'
for company in companies:
    sector, ws = company.split('_')[0], int(company.split('_')[1])

    print(company)
    top = 20
    lag_ws = 63
    input_path1 = f'../03 Modeling/{company}'
    input_path2 = f'../01 Raw Data/{sector}'
    output_path = f'{company}'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    full_lambda = pd.read_csv(f'{input_path1}/full_lambda.csv', index_col='Date', parse_dates=['Date'])
    FRM = full_lambda.mean(axis=1) * full_lambda.shape[1] / top

    FRM_L = pd.DataFrame()
    lagged_cols = {f'FRM_lag_{lag}': FRM.shift(lag) for lag in range(round(lag_ws+1))}
    FRM_L = pd.concat(lagged_cols, axis=1).dropna().resample('M').last()
    FRM_L = FRM_L.dropna()
    FRM_L = FRM_L.resample('M').last()

    MI_filter = MI.loc[MI.index.to_series().apply(lambda x: pd.Period(x, freq='M')).isin(FRM_L.index.to_period('M').unique())]

    #####

    X = FRM_L.copy()
    y = MI_filter['Recession']

    for col in X.columns:
        X[col] = pd.to_numeric(X[col], errors='coerce')

    cut = 5
    n = len(X)

    models = {
        "Logistic Regression": LogisticRegression(max_iter=10000, random_state=42, multi_class='ovr'),
        "XGBoost": XGBClassifier(eval_metric='mlogloss', random_state=42)
    }

    results = pd.DataFrame(columns=["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"])
    all_predictions = {model_name: pd.Series([None] * n, index=MI_filter.index) for model_name in models.keys()}

    n_samples = X.shape[0]
    split_size = n_samples // cut

    for i in tqdm(range(cut)):
        start_test = i * split_size
        end_test = (i + 1) * split_size if i < cut - 1 else n_samples

        X_test = X[start_test:end_test]
        y_test = y[start_test:end_test]

        indices = list(range(0, start_test)) + list(range(end_test, n_samples))
        X_train = X.iloc[indices]
        y_train = y.iloc[indices]

        row_results = []

        for name, model in models.items():
            model.fit(X_train, y_train)
            
            y_pred_train = model.predict(X_train)
            y_pred_test = model.predict(X_test)
            y_pred_proba_train = model.predict_proba(X_train)[:, 1]
            y_pred_proba_test = model.predict_proba(X_test)[:, 1]

            auc_score_train = roc_auc_score(y_train, y_pred_proba_train)
            brier_score_train = brier_score_loss(y_train, y_pred_proba_train)
            auc_score_test = roc_auc_score(y_test, y_pred_proba_test)
            brier_score_test = brier_score_loss(y_test, y_pred_proba_test)

            row_results.extend([round(auc_score_train, 4), round(brier_score_train, 4), round(auc_score_test, 4), round(brier_score_test, 4)])

            all_predictions[name].iloc[start_test:end_test] = y_pred_proba_test

            fig, ax = plt.subplots(figsize=(20, 4))
            color = 'darkgoldenrod' if name == 'Logistic Regression' else 'darkblue'
            ax.scatter(MI_filter.index[indices], y_train, label='Actual', color='darkred', marker='s', s=14)
            ax.plot(MI_filter.index[indices], y_pred_proba_train, label=f'Predicted by {name}', linewidth=4, color=color, alpha=0.2)
            count = 0
            for idx in indices:
                if y_train.loc[MI_filter.index[idx]] == 1 and y_pred_proba_train[count] < 0.5:
                    ax.scatter(MI_filter.index[idx], y_pred_proba_train[count], color='darkgreen', marker='x', s=50, lw=2.5, alpha=1)
                elif y_train.loc[MI_filter.index[idx]] == 0 and y_pred_proba_train[count] > 0.5:
                    ax.scatter(MI_filter.index[idx], y_pred_proba_train[count], color='darkgreen', marker='^', s=40, lw=1, alpha=1)
                else:
                    ax.scatter(MI_filter.index[idx], y_pred_proba_train[count], color=color, alpha=1)
                count += 1
            ax.axhline(y=0.5, color='k', linestyle='--', alpha=0.4)
            ax.set_xlim(y.index[0], y.index[-1])
            ax.set_ylim(-0.1, 1.1)
            ax.tick_params(axis='y', labelsize=8, width=2)
            ax.tick_params(axis='x', labelsize=8, width=2)
            ax.xaxis.set_major_locator(mdates.YearLocator())
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
            plt.tight_layout()
            plt.savefig(f'{output_path}/IN FRM {feature} {name} F{i+1}.png', dpi=144, transparent=True)
            plt.close()

        results.loc[i] = row_results

    results.columns = ["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"]

    average_results = results.mean().to_frame().T
    average_results.index = ["-"]

    print(results.round(4))
    print(average_results.round(4))
    print()

    for name in models.keys():
        plt.figure(figsize=(20, 4))
        color = 'darkgoldenrod' if name == 'Logistic Regression' else 'darkblue'
        plt.scatter(MI_filter.index, y, label='Actual', color='darkred', marker='s', s=14)
        # plt.scatter(MI_filter.index, all_predictions[name], label=f'Predicted by {name}', s=20)
        plt.plot(MI_filter.index, all_predictions[name], label=f'Predicted by {name}', linewidth=4, color=color, alpha=0.2)
        for i in range(len(all_predictions[name])):
            if y.iloc[i] == 1 and all_predictions[name].values[i] < 0.5:
                plt.scatter(all_predictions[name].index[i], all_predictions[name].values[i], color='darkgreen', marker='x', s=50, lw=2.5, alpha=1)
            elif y.iloc[i] == 0 and all_predictions[name].values[i] > 0.5:
                plt.scatter(all_predictions[name].index[i], all_predictions[name].values[i], color='darkgreen', marker='^', s=40, lw=1, alpha=1)
            else:
                plt.scatter(all_predictions[name].index[i], all_predictions[name].values[i], color=color, alpha=1)

        plt.axhline(y=0.5, color='k', linestyle='--', alpha=0.4)
        plt.ylim(-0.1, 1.1)
        plt.yticks(fontsize=14, fontweight='heavy')
        plt.xticks(fontsize=14, fontweight='heavy')
        plt.gca().xaxis.set_major_locator(mdates.YearLocator())
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        plt.tight_layout()
        plt.savefig(f'{output_path}/OUT FRM {feature} {name}.png', dpi=144, transparent=True)
        plt.close()

# FRM, with Feature selection

In [ ]:
companies = ['Cap_63', 'Cap_126', 'Ele_63', 'Ele_126', 'Fin_63', 'Fin_126', 'FinEle_63', 'FinEle_126']
company = 'Cap_63'
for company in companies:
    sector, ws = company.split('_')[0], int(company.split('_')[1])

    print(company)
    top = 20
    lag_ws = 63
    input_path1 = f'../03 Modeling/{company}'
    input_path2 = f'../01 Raw Data/{sector}'
    output_path = f'{company}'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    full_lambda = pd.read_csv(f'{input_path1}/full_lambda.csv', index_col='Date', parse_dates=['Date'])
    FRM = full_lambda.mean(axis=1) * full_lambda.shape[1] / top

    FRM_L = pd.DataFrame()
    lagged_cols = {f'FRM_lag_{lag}': FRM.shift(lag) for lag in range(round(lag_ws+1))}
    FRM_L = pd.concat(lagged_cols, axis=1).dropna().resample('M').last()
    FRM_L = FRM_L.dropna()
    FRM_L = FRM_L.resample('M').last()

    MI_filter = MI.loc[MI.index.to_series().apply(lambda x: pd.Period(x, freq='M')).isin(FRM_L.index.to_period('M').unique())]

    #####

    X = FRM_L.copy()
    y = MI_filter['Recession']

    for col in X.columns:
        X[col] = pd.to_numeric(X[col], errors='coerce')

    cut = 5
    n = len(X)

    models = {
        "Logistic Regression": LogisticRegression(max_iter=10000, random_state=42, multi_class='ovr'),
        "XGBoost": XGBClassifier(eval_metric='mlogloss', random_state=42)
    }

    results = pd.DataFrame(columns=["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"])
    all_predictions = {model_name: pd.Series([None] * n, index=MI_filter.index) for model_name in models.keys()}

    n_samples = X.shape[0]
    split_size = n_samples // cut

    for i in tqdm(range(cut)):
        start_test = i * split_size
        end_test = (i + 1) * split_size if i < cut - 1 else n_samples

        X_test = X[start_test:end_test]
        y_test = y[start_test:end_test]

        indices = list(range(0, start_test)) + list(range(end_test, n_samples))
        X_train = X.iloc[indices]
        y_train = y.iloc[indices]

        lasso_cv = LassoCV(cv=10, random_state=42, max_iter=10000, n_jobs=-1).fit(X_train, y_train)
        best_alpha = lasso_cv.alpha_
        lasso = Lasso(alpha=best_alpha)
        lasso.fit(X_train, y_train)

        selected_features = np.where(lasso.coef_ != 0)[0]

        count = 0
        while len(selected_features) < top:
            count += 1
            lasso = Lasso(alpha=best_alpha*(0.5**count))
            lasso.fit(X_train, y_train)
            selected_features = np.where(lasso.coef_ != 0)[0]

        X_train = X_train.iloc[:, selected_features]
        X_test = X_test.iloc[:, selected_features]

        row_results = []

        for name, model in models.items():
            model.fit(X_train, y_train)
            
            y_pred_train = model.predict(X_train)
            y_pred_test = model.predict(X_test)
            y_pred_proba_train = model.predict_proba(X_train)[:, 1]
            y_pred_proba_test = model.predict_proba(X_test)[:, 1]

            auc_score_train = roc_auc_score(y_train, y_pred_proba_train)
            brier_score_train = brier_score_loss(y_train, y_pred_proba_train)
            auc_score_test = roc_auc_score(y_test, y_pred_proba_test)
            brier_score_test = brier_score_loss(y_test, y_pred_proba_test)

            row_results.extend([round(auc_score_train, 4), round(brier_score_train, 4), round(auc_score_test, 4), round(brier_score_test, 4)])

            all_predictions[name].iloc[start_test:end_test] = y_pred_proba_test

        results.loc[i] = row_results

    results.columns = ["LR_AUC", "LR_Brier", "LR_AUC", "LR_Brier", "XGB_AUC", "XGB_Brier", "XGB_AUC", "XGB_Brier"]

    average_results = results.mean().to_frame().T
    average_results.index = ["-"]

    print(results.round(4))
    print(average_results.round(4))
    print()


In [ ]:
companies = ['Cap_63', 'Cap_126', 'Ele_63', 'Ele_126', 'Fin_63', 'Fin_126', 'FinEle_63', 'FinEle_126']

for company in companies:
    sector, ws = company.split('_')[0], int(company.split('_')[1])

    print(company)
    top = 20
    lag_ws = 63
    input_path1 = f'../03 Modeling/{company}'
    input_path2 = f'../01 Raw Data/{sector}'
    output_path = f'{company}'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    full_lambda = pd.read_csv(f'{input_path1}/full_lambda.csv', index_col='Date', parse_dates=['Date'])
    stock_rank = pd.read_csv(f'{input_path2}/Top rank company.csv', index_col='Date', parse_dates=['Date'])
    stock_rank = stock_rank.astype(str)

    lambda_rank = pd.DataFrame(index=full_lambda.index, columns=range(1, 21))
    for date in full_lambda.index:
        for col in lambda_rank.columns:
            company_id = str(stock_rank.loc[date, str(col)]).split('.')[0]
            lambda_rank.loc[date, col] = full_lambda.loc[date, company_id]
    scaler = MinMaxScaler()
    lambda_rank[:] = scaler.fit_transform(lambda_rank.values)
    lambda_rank_L = pd.DataFrame()
    for column in lambda_rank:
        lagged_cols = {f'{column}_{lag}': lambda_rank.loc[:,column].shift(lag) for lag in range(round(lag_ws+1))}
        lagged = pd.concat(lagged_cols, axis=1).dropna().resample('M').last()
        lambda_rank_L = pd.concat([lambda_rank_L, lagged], axis=1)

    MI_filter = MI.loc[MI.index.to_series().apply(lambda x: pd.Period(x, freq='M')).isin(lambda_rank_L.index.to_period('M').unique())]

    X = lambda_rank_L.copy()
    y = MI_filter['Recession']

    for col in X.columns:
        X[col] = pd.to_numeric(X[col], errors='coerce')

    cut = 5
    n = len(X)

    selected_features_list = []
    total_features_list = []

    n_samples = X.shape[0]
    split_size = n_samples // cut

    for i in tqdm(range(cut)):
        start_test = i * split_size
        end_test = (i + 1) * split_size if i < cut - 1 else n_samples

        X_test = X[start_test:end_test]
        y_test = y[start_test:end_test]

        indices = list(range(0, start_test)) + list(range(end_test, n_samples))
        X_train = X.iloc[indices]
        y_train = y.iloc[indices]

        lasso_cv = LassoCV(cv=10, random_state=42, max_iter=10000, n_jobs=-1).fit(X_train, y_train)
        best_alpha = lasso_cv.alpha_
        lasso = Lasso(alpha=best_alpha)
        lasso.fit(X_train, y_train)

        selected_features = np.where(lasso.coef_ != 0)[0]

        count = 0
        while len(selected_features) < top:
            count += 1
            lasso = Lasso(alpha=best_alpha*(0.5**count))
            lasso.fit(X_train, y_train)
            selected_features = np.where(lasso.coef_ != 0)[0]

        selected_feature_names = X_train.columns[selected_features]
        selected_features_list.append(selected_feature_names)
        total_features_list.append(len(selected_feature_names))

    for i, features in enumerate(selected_features_list):
        print(f'Total: {total_features_list[i]}')
        print(f"Fold {i+1}: {features.tolist()}")
